In [ ]:
import sys
sys.path.append("/data1/andrew/meng/mixehr/meng/VAE-EHR/src")
from importlib import reload

import numpy as np 
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn, optim

In [ ]:
import vae
reload(vae)

from vae import VAE, VAETrainer

In [ ]:
patient_icd_file = "/data1/andrew/meng/mixehr/data/Mimic/andrew_outputs/PATIENT_ICD_BINARY.csv"
patient_icd_df = pd.read_csv(patient_icd_file, sep=' ')

patient_icd_data = patient_icd_df.drop('SUBJECT_ID', axis=1)

data = torch.tensor(patient_icd_data.values).float()
print(data.shape)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print((device))

print("Feature_dim: {}".format(data.shape[1]))
model = VAE(
    feature_dim = data.shape[1], 
    encoder_dim = [(250, 500), (500, 250), (250, 100)],
    latent_dim = 10,
    decoder_dim = [],
    use_relu=True
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
print(type(optimizer))

In [ ]:
date="200110"
experiment_name = "{}_patient_clusters_relu_encoder_single_layer_decoder".format(date)
trainer = VAETrainer(
    model=model, 
    device=device, 
    optimizer=optimizer,
    experiment_name=experiment_name
)

In [ ]:
trainer.train(
    data=data, 
    epochs=40,
    batch_size=40,
    save_model_interval=5,
    clip_gradients=False
)

In [ ]:
#model.load_state_dict(torch.load("VAE_epoch_450.pkl"))